# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686564


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:25,  1.11it/s]

Rendering models:  10%|█         | 3/30 [00:06<00:57,  2.13s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:39,  1.53s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:27,  1.11s/it]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:01,  6.65it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  7.33it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  8.11it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  9.18it/s]

CThomas                               0.051653
not-logged-in-f7e2260ed4a9517a3e4a    0.001501
cschwefl                              0.072509
not-logged-in-975ec4b89e5ce0be836b    0.000297
yooman12                              0.000350
Camelron                              0.000378
ellianderjoy                          0.006825
djswanso                              0.000502
not-logged-in-83c90d58182a261c0405    0.002795
Brandan                               0.001725
not-logged-in-5ba79dfc2e6bd33509ae    0.042015
Jessica_Korkmaz                       0.010623
not-logged-in-5f30b2bb9b7f9d49da41    0.005698
not-logged-in-5f30b2bb9b7f9d49da41    0.005085
raisamaduro                           0.000904
not-logged-in-09330e0b15263888ee9d    0.001776
tullmejs                              0.000475
DarioB                                0.000787
not-logged-in-24db7619b192ada33b8d    0.053061
JulieHedglin                          0.000399
Saharisunshine                        0.001921
RaulE        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())